# Eager Few Shot Object Detection Colab

Welcome to the Eager Few Shot Object Detection Colab --- in this colab we demonstrate fine tuning of a (TF2 friendly) RetinaNet architecture on very few examples of a novel class after initializing from a pre-trained COCO checkpoint.
Training runs in eager mode.

Estimated time to run through this colab (with GPU): < 5 minutes.

## Imports

In [ ]:
# cd  "C:\Users\sky66\Downloads\models\research" (cd to this folder)
# jupyter notebook

# conda install tensorflow=2.5.0=gpu_py39h7dc34a2_0
# conda uninstall tensorflow
# pip install "tensorflow==2.7.0"

# git clone --depth 1 https://github.com/tensorflow/models

# pip uninstall pyparsing -y
# pip install pyparsing==2.4.2

# pip uninstall pyyaml -y
# pip install pyyaml==5.1

# protoc object_detection/protos/*.proto --python_out=.
# python object_detection/packages/tf2/setup.py build
# python object_detection/packages/tf2/setup.py install

# conda list

# 參考資料
# https://www.jianshu.com/p/f8ffbf18c312
# https://stackoverflow.com/questions/68737130/error-while-import-keras-attributeerror-module-tensorflow-compat-v2-interna
# https://github.com/tensorflow/tensorflow/issues/53060
# https://medium.com/ching-i/win10-%E5%AE%89%E8%A3%9D-cuda-cudnn-%E6%95%99%E5%AD%B8-c617b3b76deb
# https://github.com/tensorflow/tensorflow/issues/52988
# https://cppsecrets.com/users/17211410511511610510710997106117109100971144964103109971051084699111109/Python-tqdmsetpostfixstr.php
# https://www.delftstack.com/zh-tw/howto/python/python-print-flush/

# 將資料分成驗證和訓練用

In [18]:
# !git clone https://github.com/datitran/raccoon_dataset.git

In [1]:
import numpy as np
import os
import shutil

In [2]:
# 取得用 Get_BBox.ipynb 創建的 bboxy 資料
name = []
bbox = []
with open("bbox.csv", "r", encoding="utf8") as f:
    for e in f:
        e = e.strip()
        d = e.split(",")
        name.append(d[0])
        bbox.append(np.array(d[1:], dtype="float16"))

In [ ]:
# 80% 訓練用，20%驗證用。
train = round(len(name)*0.8)
test = len(name)-train

In [3]:
path = r"C:\Users\sky66\fiftyone\coco-2017\raw\nlp" # 原始圖像資料夾，圖片由 Get_BBox.ipynb 下載、創建
tp = r"C:\Users\sky66\fiftyone\coco-2017\raw\train" # train path
vp = r"C:\Users\sky66\fiftyone\coco-2017\raw\test" # val path

In [ ]:
# 將圖像由原始資料夾移至新訓練用或驗證用資料夾

In [ ]:
f = list(os.listdir(path))

In [ ]:
for i  in range(test):
    s = np.random.randint(0,len(f))
    n = f.pop(s)
    shutil.move(path+"\\"+n, vp+"\\"+n)

In [ ]:
for i  in range(train):
    n = f.pop(0)
    shutil.move(path+"\\"+n, tp+"\\"+n)

In [ ]:
# 準備資料建立tfrecord

In [4]:
f1 = list(os.listdir(tp))
f2 = list(os.listdir(vp))
bt = []
bv = []

In [5]:
top = ",".join(["filename", "width", "height", "class", "xmin", "ymin", "xmax", "ymax"])+"\n"

In [15]:
for n, b in zip(name, bbox):
    # ymin, xmin, ymax, xmax = b.tolist()
    s = [n, 640, 640, "person", *b.tolist()]
    if n in f1:
        bt.append(s)
    else:
        bv.append(s)

In [16]:
bt[0]

['train_000000000036.jpg',
 640,
 640,
 'person',
 0.25439453125,
 0.348388671875,
 0.9814453125,
 0.994140625]

# 建立TFRecord

In [17]:
import os
import io
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [18]:
def create_tf_example(bbox, path):
    n, w, h, C, ymin, xmin, ymax, xmax = bbox
    with tf.io.gfile.GFile(os.path.join(path, n), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)


    image_format = b'jpg'
    xmins = [xmin]
    xmaxs = [xmax]
    ymins = [ymin]
    ymaxs = [ymax]
    classes_text = [bytes(C, "utf8")]
    classes = [1]
    width = w
    height = h
    filename = bytes(n, "utf8")

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [19]:
op = "train.tfrecord"
writer = tf.io.TFRecordWriter(os.getcwd()+"\\"+op)

for group in bt:
    tf_example = create_tf_example(group, tp)
    writer.write(tf_example.SerializeToString())

writer.close()

In [20]:
op = "test.tfrecord"
writer = tf.io.TFRecordWriter(os.getcwd()+"\\"+op)

for group in bv:
    tf_example = create_tf_example(group, vp)
    writer.write(tf_example.SerializeToString())

writer.close()

# 準備訓練用檔案

In [ ]:
# move tfrecord , config, checkpoint, pbtxt.

In [ ]:
# https://blog.csdn.net/weixin_44238733/article/details/109861924

In [24]:
S = "item{\n  id: 1\n  name: 'person'\n}"

with open("train.pbtxt", "w", encoding="utf8") as f:
    f.write(S)
    
with open("test.pbtxt", "w", encoding="utf8") as f:
    f.write(S)    

In [25]:
print(S)

item{
  id: 1
  name: 'person'
}


In [ ]:
# 修改config

In [ ]:
# pip install typeguard

In [ ]:
# python "C:/Users/sky66/Downloads/models/research/object_detection/model_main_tf2.py" --pipeline_config_path="C:/Users/sky66/Downloads/models/research/my_model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config" --model_dir="C:/Users/sky66/Downloads/models/research/my_model/new_model" --alsologtostderr